# Write discharge records
### ⚠️ Writes only records part (for two sediments) of `.bct` or `.bcc` files! ⚠️ 

#### Required input 
* timestep
* start and end of discharge time
* Number of sigma layers
* Number of sigma layer over which boundary condition is imposed
* Sediment weight per volume [kg/m3]


## TODO
* Split into a function
    * Multiple sediment + multiple times!
* Use pandas
* Don't hardcode end time as string
* Read vertical grid from .mdf and .dep file
* Read horizontal cell size from grid file
* Plot time vs discharge of boundary condition
* Make more modular to allow for more than two sediments
    * Make a function for writing records to new file
* Add some plotting of the column to visualize height of layers and among which layers flow is imposed
* Write the whole .bcc/.bct files instead of just the records
* Read values from mdf and dep file

## Might be good
1. Use [Pint](https://pint.readthedocs.io/en/0.11/) to keep track of units
```python
import pint
ureg = pint.UnitRegistry()
```

In [2]:
import os
import numpy as np
from JulesD3D.utils import formatSci
from IPython.display import Markdown as md

# Input

In [3]:
nr_sigma_layers = 80 # total sigma layers
nr_discharge_layers = 53
nr_zero_discharge_layers = nr_sigma_layers - nr_discharge_layers

In [4]:
sand_mass_conc = 19.875 # [kg/m3] #  # 25% 39.74
silt_mass_conc = 59.625 # [kg/m3] #  # 75% 39.74

In [5]:
cell_width = 200   # ureg.meter [m] 
cell_length = 200  # ureg.meter [m]
init_depth = 300   # ureg.meter [m]
density_sed = 2650 # [kg/m3]

In [6]:
discharge_total = 4500 # [m3/s]

In [7]:
timestep = 0.3 # [min]
start_discharge_time = 9 # [min]
discharge_duration = 39 # [min]

end_time = 1230
formatSci(end_time)

'1.2300000e+003'

In [8]:
nr_of_timesteps = int(end_time / timestep)
nr_of_timesteps

In [24]:
discharge_duration

39

---

In [25]:
end_discharge_time = start_discharge_time + discharge_duration # [min]

#### Discharge is linearly interpolated between timesteps
Therefore, we need to add one timestep before the start and one timestep after in which discharges are zero

In [8]:
pre_start_discharge_time = start_discharge_time - timestep  # [min]
post_end_discharge_time = end_discharge_time + timestep  # [min]

| ⚠️ WARNING: Always check wether time is a multiple of timestep! ⚠️ |
| --- |

In [43]:
from decimal import Decimal
md(f'''
# Sanity check
Weird check for floating point errors
## Is discharge duration a multiple of timestep? {Decimal(discharge_duration) % Decimal(timestep) < 10e-9}!
''') 


# Sanity check
Weird check for floating point errors
## Is discharge duration a multiple of timestep? True!


In [35]:
# should be read from mdf file
sigma_layers_percentages = np.array([7.180, 6.000, 5.500, 5.000, 4.500, 4.000, 3.800, 3.600, 3.400, 3.200, 3.000, 2.900, 2.800, 2.700, 2.600, 2.500, 2.400, 2.300, 2.200, 2.100, 2.000, 1.900, 1.800, 1.700, 1.600, 1.500, 1.400, 1.300, 1.200, 1.100, 1.000, 0.950, 0.900, 0.850, 0.800, 0.750, 0.700, 0.650, 0.600, 0.550, 0.500, 0.450, 0.400, 0.350, 0.300, 0.270, 0.230, 0.200, 0.150, 0.130, 0.120, 0.110, 0.100, 0.100, 0.090, 0.090, 0.090, 0.090, 0.090, 0.080, 0.080, 0.080, 0.080, 0.070, 0.070, 0.070, 0.060, 0.060, 0.060, 0.060, 0.060, 0.050, 0.050, 0.050, 0.050, 0.040, 0.040, 0.040, 0.030, 0.03])

In [11]:
# TODO: hardcoded simultion start + edn time string
times = [f"{formatSci(0)}", 
         f"{formatSci(pre_start_discharge_time)}", 
         f"{formatSci(start_discharge_time)}",\
         f"{formatSci(end_discharge_time)}", 
         f"{formatSci(post_end_discharge_time)}", 
         formatSci(end_time)] # "1.2300000e+003"]
times

['0.0000000e+000',
 '8.7000000e+000',
 '9.0000000e+000',
 '4.8000000e+001',
 '4.8300000e+001',
 '1.2300000e+003']

## Write records

In [12]:
all_zeros_line = '  '.join('0.0000000e+000' for i in range(nr_sigma_layers))
line_all_zero = f"{all_zeros_line}  {all_zeros_line}" # A + B

In [13]:
# Repeat '0.0000000e+000' nr_zero_discharge_layers times
# np.zeroes(zero_records_discharge_layer)
zero_records_discharge_layer = '  '.join('0.0000000e+000' for i in range(nr_zero_discharge_layers))

# Write Concentration file (.bcc)

In [14]:
formatted_sand_conc = formatSci(sand_mass_conc)
formatted_silt_conc = formatSci(silt_mass_conc)

In [15]:
md(f"### The first {nr_zero_discharge_layers} 'cells' are all zeroes, then the next {nr_discharge_layers} layers contain the discharge values")

### The first 27 'cells' are all zeroes, then the next 53 layers contain the discharge values

In [16]:
# TODO: should loop over arrray of sediments and then write each line but for only two like now it works fine
sand_conc_records_discharge_layer = '  '.join(formatted_sand_conc for i in range(nr_discharge_layers))
silt_conc_records_discharge_layer = '  '.join(formatted_silt_conc for i in range(nr_discharge_layers))

discharge_sand_vol_line = f"{zero_records_discharge_layer}  {sand_conc_records_discharge_layer}"
discharge_silt_vol_line = f"{zero_records_discharge_layer}  {silt_conc_records_discharge_layer}"

# Each line has two discharge 'points'
line_with_sand_conc_records = f'{discharge_sand_vol_line}  {discharge_sand_vol_line}' # A + B
line_with_silt_conc_records = f'{discharge_silt_vol_line}  {discharge_silt_vol_line}' # A + B
bcc_filename = './generated/7525_15.bccrecords'

# time + A + B
with open(bcc_filename, 'w') as f:
    # why not use a f multiline string here f'''{ line below} '''
    linesToWrite = [
        "sand\n",
        f" {times[0]}  {line_all_zero}" + '\n',
        f" {times[1]}  {line_all_zero}" + '\n',
        f" {times[2]}  {line_with_sand_conc_records}" + '\n',
        f" {times[3]}  {line_with_sand_conc_records}" + '\n',
        f" {times[4]}  {line_all_zero}" + '\n',
        f" {times[5]}  {line_all_zero}" + '\n',
        "\n",
        "silt\n"
        f" {times[0]}  {line_all_zero}" + '\n',
        f" {times[1]}  {line_all_zero}" + '\n',
        f" {times[2]}  {line_with_silt_conc_records}" + '\n',
        f" {times[3]}  {line_with_silt_conc_records}" + '\n',
        f" {times[4]}  {line_all_zero}" + '\n',
        f" {times[5]}  {line_all_zero}" + '\n'
    ]
    f.writelines(line for line in linesToWrite)

# Write Discharge file (.bct)

In [17]:
discharge_layer_sigma_percentages = sigma_layers_percentages[nr_zero_discharge_layers:]

In [18]:
discharge_layers_sigma_percentage_sum = discharge_layer_sigma_percentages.sum()
md(f"Discharge layers: {discharge_layers_sigma_percentage_sum}%")

Discharge layers: 16.42%

In [19]:
md(f"Now divide {discharge_total} $m^3/s$ discharge among the bottom {discharge_layers_sigma_percentage_sum}% of sigma layers")

Now divide 4500 $m^3/s$ discharge among the bottom 16.42% of sigma layers

In [20]:
new_discharge_layers = discharge_layer_sigma_percentages/discharge_layers_sigma_percentage_sum * discharge_total

In [21]:
correct = discharge_total == new_discharge_layers.sum()

md(f'''Check if its right:
    Sum of discharges over {nr_discharge_layers} discharge layers is {new_discharge_layers.sum()} $m^3/s$
    and entered discharge is {discharge_total} $m^3/s$, so it's {"**correct!**" if correct else "**not right man**"}
    ''')


Check if its right:
    Sum of discharges over 53 discharge layers is 4499.999999999998 $m^3/s$
    and entered discharge is 4500 $m^3/s$, so it's **not right**
    

In [22]:
discharge_records_discharge_layer = '  '.join(formatSci(discharge) for discharge in new_discharge_layers)

In [23]:
discharge_line_formatted = f"{zero_records_discharge_layer}  {discharge_records_discharge_layer}"
# Each line has two discharge 'points'
line_with_records = f'{discharge_line_formatted}  {discharge_line_formatted}' # A + B
bct_filename = f'./generated/discharge{discharge_total}.bctrecords'

# Format: time + A + B
with open(bct_filename, 'w') as f:
    linesToWrite = [
        f" {times[0]}  {line_all_zero}\n",
        f" {times[1]}  {line_all_zero}\n",
        f" {times[2]}  {line_with_records}\n",
        f" {times[3]}  {line_with_records}\n",
        f" {times[4]}  {line_all_zero}\n",
        f" {times[5]}  {line_all_zero}\n"
    ]
    f.writelines(line for line in linesToWrite)

In [24]:
# should have concat'ed the values like this first then formatting instead of joining formatted stuff
discharge_line_values = np.hstack([np.zeros(nr_zero_discharge_layers), new_discharge_layers]) # m3/s

# Calculate a bunch-o-stuff

In [25]:
cell_heights = np.array(sigma_layers_percentages)/100 * init_depth # [m]
cell_volumes = cell_heights * cell_width * cell_length # [m3]

## Calculate concentrations

#### Calculate total sand volume in column

In [26]:
sand_zero_dis_layer = np.zeros(nr_zero_discharge_layers) # [kg/m3]
sand_dis_layer = np.ones(nr_discharge_layers) * sand_mass_conc # [kg/m3]

all_sand_layers_weight_vol = np.hstack([sand_zero_dis_layer, sand_dis_layer]) # [kg/m3]
vol_sand_per_layer = all_sand_layers_weight_vol * cell_volumes / density_sed # [m3]
total_sand_vol_in_column = vol_sand_per_layer.sum()

#### Calculate total silt volume in column

In [27]:
silt_zero_dis_layer = np.zeros(nr_zero_discharge_layers)
silt_dis_layer = np.ones(nr_discharge_layers) * silt_mass_conc

all_silt_layers_weight_vol = np.hstack([silt_zero_dis_layer, silt_dis_layer]) # [kg/m3]
vol_silt_per_layer = all_silt_layers_weight_vol * cell_volumes / density_sed # [m3]
total_silt_vol_in_column = vol_silt_per_layer.sum()

In [28]:
total_vol_discharge_cells = cell_volumes[nr_zero_discharge_layers:].sum() # [m3]

In [29]:
sand_conc = total_sand_vol_in_column/total_vol_discharge_cells
silt_conc = total_silt_vol_in_column/total_vol_discharge_cells

In [30]:
total_sediment_conc = (silt_conc+sand_conc)

## Calculate total sediment weight and volume coming in
$[m^3/s] \times [kg/m^3] \times [s] = [kg]$

discharge $\times$ weight volume $\times$ duration

In [31]:
sand_weight_flowing_in = discharge_line_values * all_sand_layers_weight_vol * discharge_duration * 60 # LIST [m3/s] * [kg/m3] * [s] = [kg] 
total_sand_weight_flowing_in = sand_weight_flowing_in.sum() # [kg]

In [32]:
silt_weight_flowing_in = discharge_line_values * all_silt_layers_weight_vol * discharge_duration * 60 # LIST [m3/s] * [kg/m3] * [s] = [kg] 
total_silt_weight_flowing_in = silt_weight_flowing_in.sum() # [kg]

In [33]:
total_sediment_weight_flowing_in = total_sand_weight_flowing_in + total_silt_weight_flowing_in
total_sediment_volume_flowing_in = total_sediment_weight_flowing_in/density_sed

In [34]:
md(f'''
* **Total sediment volume flowing in = {total_sediment_volume_flowing_in/10**5} $\\times10^5 m^3$**
''')


* **Total sediment volume flowing in = 3.159 $\times10^5 m^3$**


<hr>

In [35]:
total_discharge_volume = discharge_duration * 60 * discharge_total # [s * m3/s = m3]

In [52]:
# turn this into a markdown table
md(f'''
# Summary

### Time
* Total simulated time = {int(end_time)/60} hours
* Number of timesteps = {nr_of_timesteps}



### Flow
* Total flow discharge volume = {total_discharge_volume/10**6} $ \\times 10^6 m^3$
* Discharge height = {cell_heights[nr_zero_discharge_layers:].sum()} m
* Total discharge = {discharge_total} $m^3/s$

* Discharge start time = {start_discharge_time}
* Discharge end time = {end_discharge_time}
* Discharge duration = {end_discharge_time - start_discharge_time} min

### Sediments
* Silt weight per volume = {silt_mass_conc} $kg/m^3$
* Sand weight per volume = {sand_mass_conc} $kg/m^3$

* Total silt volume {round(total_sand_weight_flowing_in/density_sed/10**5, 2)} $\\times10^5 m^3$
* Total sand volume {round(total_silt_weight_flowing_in/density_sed/10**5, 2)} $\\times10^5 m^3$
* **Total sediment volume = {round(total_sediment_volume_flowing_in/10**5, 3)} $\\times10^5 m^3$**

* Silt Concentration = {round(silt_conc * 100, 2)}%
* Sand Concentration =  {round(sand_conc * 100, 2)}%
* Total sediment concentration = {round(total_sediment_conc * 100, 2)}%

''')


# Summary

### Time
* Total simulated time = 20.5 hours
* Number of timesteps = 4100



### Flow
* Total flow discharge volume = 10.53 $ \times 10^6 m^3$
* Discharge height = 49.26 m
* Total discharge = 4500 $m^3/s$

* Discharge start time = 9
* Discharge end time = 48
* Discharge duration = 39 min

### Sediments
* Silt weight per volume = 59.625 $kg/m^3$
* Sand weight per volume = 19.875 $kg/m^3$

* Total silt volume 0.79 $\times10^5 m^3$
* Total sand volume 2.37 $\times10^5 m^3$
* **Total sediment volume = 3.159 $\times10^5 m^3$**

* Silt Concentration = 2.25%
* Sand Concentration =  0.75%
* Total sediment concentration = 3.0%



In [31]:
# shouldnt it get the sigma stuff from an mdf class or something?
class MonsterBCCClass():
    def __init__(self, nr_sigma_layers, nr_discharge_layers):
        self.nr_sigma_layers = nr_sigma_layers
        self.nr_discharge_layers = nr_discharge_layers
        self.nr_zero_discharge_layers = nr_sigma_layers - nr_discharge_layers
        # self.sigma_percentage = sigma_percentage
        
        print(self.nr_zero_discharge_layers)
MonsterBCCClass(80, 57)

23


In [17]:
# # mass conc is just repeated nr_discharge_layers times
# # whereas discharge values are cell specific 
# def makeDischargeRecords(mass_conc, nr_sigma_layers=80, nr_discharge_layers = 53):
#     # remove to class
#     nr_zero_discharge_layers = nr_sigma_layers - nr_discharge_layers    
#     formatted_mass_conc = formatSci(mass_conc)
#     #####
#     print(f"The first {nr_zero_discharge_layers} 'cells' are all zeroes, then the next {nr_discharge_layers} layers contain the discharge values: {mass_conc}")
    
#     zero_records_discharge_layer = '  '.join('0.0000000e+000' for i in range(nr_zero_discharge_layers))
    
#     # what if discharge layers are an argument to this function?
#     records_discharge_layers = '  '.join(formatted_mass_conc for i in range(nr_discharge_layers))
    
#     # first the cells where discharge is zero, then the cells that discharge 
#     discharge_line = f"{zero_records_discharge_layer}  {records_discharge_layers}"

#     # Each line has two discharge 'points' so repeat it twice
#     complete_line_conc_records = f'{discharge_line}  {discharge_line}' # A + B

#     return complete_line_conc_records

In [ ]:
def allZeroesRecords(nr_sigma_layers=80):
    all_zeros_line = '  '.join('0.0000000e+000' for i in range(nr_sigma_layers))
    line_all_zero = f"{all_zeros_line}  {all_zeros_line}" # left + right
    
    # Repeat '0.0000000e+000' nr_zero_discharge_layers times
    zero_records_discharge_layer = '  '.join('0.0000000e+000' for i in range(nr_zero_discharge_layers))
    
    return zero_records_discharge_layer

In [27]:
def makeBcTimes(timestep=0.3, start_discharge_time=15, discharge_duration=45):
    # Discharge is linearly interpolated between timesteps
    # Therefore, we need to add one timestep before the start and one timestep after in which discharges are zero
    end_discharge_time = start_discharge_time + discharge_duration # [min]
    pre_start_discharge_time = start_discharge_time - timestep  # [min]
    post_end_discharge_time = end_discharge_time + timestep  # [min]    

    times = [formatSci(0), 
             formatSci(pre_start_discharge_time), 
             formatSci(start_discharge_time),
             formatSci(end_discharge_time), 
             formatSci(post_end_discharge_time), 
             formatSci(end_time)
            ]
    
    return times

In [51]:
def calcDischargePerCell(discharge_total=4500, sigma_layers_percentages=[], nr_zero_discharge_layers=0):
    discharge_layer_sigma_percentages = sigma_layers_percentages[nr_zero_discharge_layers:] # select only the discharge parts
    discharge_layers_sigma_percentage_sum = discharge_layer_sigma_percentages.sum() # sum it

    print(f"Now divide {discharge_total} $m^3/s$ discharge among the bottom {discharge_layers_sigma_percentage_sum}% height of sigma layers")
    new_discharge_layers = discharge_layer_sigma_percentages/discharge_layers_sigma_percentage_sum * discharge_total
    
    discharge_records = '  '.join(formatSci(discharge) for discharge in new_discharge_layers)

    return discharge_records

In [47]:
def makeDischargeRecords(discharge_records=[], nr_sigma_layers=80, nr_discharge_layers = 53):
    # remove to class
    nr_zero_discharge_layers = nr_sigma_layers - nr_discharge_layers    

    print(f"The first {nr_zero_discharge_layers} 'cells' are all zeroes, then the next {nr_discharge_layers} layers contain the discharge values")
    
    zero_records_discharge_layer = '  '.join('0.0000000e+000' for i in range(nr_zero_discharge_layers))
    
    # first the cells where discharge is zero, then the cells that discharge 
    discharge_line = f"{zero_records_discharge_layer}  {discharge_records}"

    # Each line has two discharge 'points' so repeat it twice
    complete_line_conc_records = f'{discharge_line}  {discharge_line}' # A + B

    return complete_line_conc_records

In [52]:
calcDischargePerCell(sigma_layers_percentages, nr_zero_discharge_layers=nr_zero_discharge_layers)

Now divide 4500 $m^3/s$ discharge among the bottom 16.42% height of sigma layers


'3.5627284e+002  3.2886724e+002  3.0146163e+002  2.7405603e+002  2.6035323e+002  2.4665043e+002  2.3294762e+002  2.1924482e+002  2.0554202e+002  1.9183922e+002  1.7813642e+002  1.6443362e+002  1.5073082e+002  1.3702801e+002  1.2332521e+002  1.0962241e+002  9.5919610e+001  8.2216809e+001  7.3995128e+001  6.3032887e+001  5.4811206e+001  4.1108404e+001  3.5627284e+001  3.2886724e+001  3.0146163e+001  2.7405603e+001  2.7405603e+001  2.4665043e+001  2.4665043e+001  2.4665043e+001  2.4665043e+001  2.4665043e+001  2.1924482e+001  2.1924482e+001  2.1924482e+001  2.1924482e+001  1.9183922e+001  1.9183922e+001  1.9183922e+001  1.6443362e+001  1.6443362e+001  1.6443362e+001  1.6443362e+001  1.6443362e+001  1.3702801e+001  1.3702801e+001  1.3702801e+001  1.3702801e+001  1.0962241e+001  1.0962241e+001  1.0962241e+001  8.2216809e+000  8.2216809e+000'

In [48]:
formatted_mass_conc = formatSci(sand_mass_conc)
conc_records_discharge_layers = '  '.join(formatted_mass_conc for i in range(nr_discharge_layers))
makeDischargeRecords(conc_records_discharge_layers)

The first 27 'cells' are all zeroes, then the next 53 layers contain the discharge values


'0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  0.0000000e+000  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.9875000e+001  1.98750

In [19]:
bcc_filename = './generated/7525_15.bccrecords'

In [19]:
def generateVerticalBC(filename='untitled.bccrecords', timestep = 0.3, nr_sigma_layers=80, start_discharge_time=0, discharge_duration=0, sand_mass_conc=19.75, silt_mass_conc=19.75):
    if discharge_duration == 0:
        raise Exception("discharge_duration cannot be 0")
    
    if not Decimal(discharge_duration) % Decimal(timestep) < 10e-9:
        raise Exception("Discharge duration is not a multiple of timestep")

    times = makeBcTimes(timestep=timestep, start_discharge_time=start_discharge_time, end_discharge_time=end_discharge_time)

    line_with_sand_conc_records = makeDischargeRecords(sand_mass_conc, nr_sigma_layers, nr_discharge_layers)
    line_with_silt_conc_records = makeDischargeRecords(silt_mass_conc, nr_sigma_layers, nr_discharge_layers)
    
    # --- Write Concentration file (.bcc) ---
    # time + A + B
    with open(bcc_filename, 'w') as f:
        # why not use a f multiline string here f'''{ line below} '''
        # what if more than 6 records? this approach is inflexible
        # for sediment in sediments:
            # for time in times:
            
            # one discharge always follows this sequence of 6 time values with discharge records at the 3rd and 4th place
            
        linesToWrite = [
            "sand\n",
            f" {times[0]}  {line_all_zero}" + '\n',
            f" {times[1]}  {line_all_zero}" + '\n',
            f" {times[2]}  {line_with_sand_conc_records}" + '\n',
            f" {times[3]}  {line_with_sand_conc_records}" + '\n',
            f" {times[4]}  {line_all_zero}" + '\n',
            f" {times[5]}  {line_all_zero}" + '\n',
            "\n",
            "silt\n"
            f" {times[0]}  {line_all_zero}" + '\n',
            f" {times[1]}  {line_all_zero}" + '\n',
            f" {times[2]}  {line_with_silt_conc_records}" + '\n',
            f" {times[3]}  {line_with_silt_conc_records}" + '\n',
            f" {times[4]}  {line_all_zero}" + '\n',
            f" {times[5]}  {line_all_zero}" + '\n'
        ]
        f.writelines(line for line in linesToWrite)
        
    bct_filename = f'./generated/discharge{discharge_total}.bctrecords'

    # Format: time + A + B
    with open(bct_filename, 'w') as bct_file:
        linesToWrite = [
            f" {times[0]}  {line_all_zero}\n",
            f" {times[1]}  {line_all_zero}\n",
            f" {times[2]}  {line_with_records}\n",
            f" {times[3]}  {line_with_records}\n",
            f" {times[4]}  {line_all_zero}\n",
            f" {times[5]}  {line_all_zero}\n"
        ]
        bct_file.writelines(line for line in linesToWrite) 

SyntaxError: non-default argument follows default argument (<ipython-input-19-97e09f30812f>, line 1)